In [43]:
import os
os.environ['DEBUG'] = '1'

import numpy as np

from tinygrad.tensor import Tensor
from tinygrad.nn import Linear

from lib.utils import get_mnist
from tinygrad.nn.optim import SGD

In [40]:
X_train, Y_train, X_test, Y_test = get_mnist("../data") # these need to be tensors??

In [38]:
class TinyNet:
    def __init__(self):
        self.l1 = Linear(784, 128, bias=False)
        self.l2 = Linear(128, 10, bias=False)

    def __call__(self, x):
        x = self.l1(x)
        x = x.leakyrelu()
        x = self.l2(x)
        return x

In [29]:
model = TinyNet()
criterion = Tensor.sparse_categorical_crossentropy()
opt = SGD([model.l1.weight, model.l2.weight], lr=3e-4)

In [45]:
with Tensor.train():
  for step in range(1000):
    # random sample a batch
    samp = np.random.randint(0, X_train.shape[0], size=(64))
    batch = Tensor(X_train[samp], requires_grad=False)
    # get the corresponding labels
    labels = Tensor(Y_train[samp])

    # forward pass
    out = model(batch)

    # compute loss
    loss = criterion(out, labels)

    # zero gradients
    opt.zero_grad()

    # backward pass
    loss.backward()

    # update parameters
    opt.step()

    # calculate accuracy
    pred = out.argmax(axis=-1)
    acc = (pred == labels).mean()

    if step % 100 == 0:
      print(f"Step {step+1} | Loss: {loss.numpy()} | Accuracy: {acc.numpy()}")

StopIteration: 